In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sn
from xgboost.sklearn import XGBClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, MaxPooling1D, Conv1D, AlphaDropout


import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/home/silveira/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

In [3]:
dataset = pd.read_csv('epileptic_seizure_recognition.csv')

In [4]:
tgt = dataset.y
tgt = tgt.replace([1,2,3,4,5],[0,1,1,1,1])

X = dataset.iloc[:,1:179].values
y = tgt.iloc[:].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc',
                                           min_delta=0,
                                           patience=10000,
                                           verbose=1,
                                           mode='auto')
# Initialising the ANN
clf_ann = Sequential()

# Adding the input layer and the first hidden layer
clf_ann.add(Dense(activation="relu", 
                  units=178,
                  input_dim=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the second hidden layer
clf_ann.add(Dense(activation="relu",
                  units=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the output layer
clf_ann.add(Dense(units=1,
                  kernel_initializer='glorot_normal',
                  activation="sigmoid"))

# Compiling the ANN
clf_ann.compile(optimizer=keras.optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
# loss = 'categorical_crossentropy'
# Fitting the ANN to the Training set
clf_ann.fit(X_train, y_train,
               batch_size = 256,
               epochs = 200,
               validation_data=(X_test,y_test),
               callbacks=[early_stop])

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = clf_ann.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
#cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred, target_names=['Seizure', 'Non-Seizure']))

#df_cm = pd.DataFrame(cm, range(3), range(3))
#sn.heatmap(df_cm, annot=True,fmt='g',cmap ='Blues')# font size

Train on 9200 samples, validate on 2300 samples
Epoch 1/200
9200/9200 [==============================] - 1s 146us/step - loss: 0.2149 - acc: 0.9376 - val_loss: 0.1622 - val_acc: 0.9457
Epoch 2/200
9200/9200 [==============================] - 1s 94us/step - loss: 0.1267 - acc: 0.9636 - val_loss: 0.1307 - val_acc: 0.9461
Epoch 3/200
9200/9200 [==============================] - 1s 116us/step - loss: 0.1163 - acc: 0.9654 - val_loss: 0.1077 - val_acc: 0.9583
Epoch 4/200
9200/9200 [==============================] - 1s 99us/step - loss: 0.1027 - acc: 0.9682 - val_loss: 0.1075 - val_acc: 0.9535
Epoch 5/200
9200/9200 [==============================] - 1s 111us/step - loss: 0.0971 - acc: 0.9684 - val_loss: 0.0973 - val_acc: 0.9596
Epoch 6/200
9200/9200 [==============================] - 1s 95us/step - loss: 0.0916 - acc: 0.9688 - val_loss: 0.0851 - val_acc: 0.9657
Epoch 7/200
9200/9200 [==============================] - 1s 98us/step - loss: 0.0835 - acc: 0.9727 - val_loss: 0.1003 - val_acc: 0.96

Epoch 60/200
9200/9200 [==============================] - 1s 103us/step - loss: 0.0386 - acc: 0.9857 - val_loss: 0.0553 - val_acc: 0.9813
Epoch 61/200
9200/9200 [==============================] - 1s 112us/step - loss: 0.0337 - acc: 0.9867 - val_loss: 0.0613 - val_acc: 0.9804
Epoch 62/200
9200/9200 [==============================] - 1s 117us/step - loss: 0.0369 - acc: 0.9870 - val_loss: 0.0625 - val_acc: 0.9809
Epoch 63/200
9200/9200 [==============================] - 1s 105us/step - loss: 0.0350 - acc: 0.9868 - val_loss: 0.0572 - val_acc: 0.9813
Epoch 64/200
9200/9200 [==============================] - 1s 119us/step - loss: 0.0357 - acc: 0.9870 - val_loss: 0.0575 - val_acc: 0.9813
Epoch 65/200
9200/9200 [==============================] - 1s 116us/step - loss: 0.0367 - acc: 0.9865 - val_loss: 0.0559 - val_acc: 0.9822
Epoch 66/200
9200/9200 [==============================] - 1s 101us/step - loss: 0.0347 - acc: 0.9872 - val_loss: 0.0593 - val_acc: 0.9809
Epoch 67/200
9200/9200 [==========

9200/9200 [==============================] - 1s 100us/step - loss: 0.0261 - acc: 0.9902 - val_loss: 0.0629 - val_acc: 0.9804
Epoch 120/200
9200/9200 [==============================] - 1s 98us/step - loss: 0.0264 - acc: 0.9897 - val_loss: 0.0588 - val_acc: 0.9809
Epoch 121/200
9200/9200 [==============================] - 1s 121us/step - loss: 0.0239 - acc: 0.9912 - val_loss: 0.0600 - val_acc: 0.9791
Epoch 122/200
9200/9200 [==============================] - 1s 139us/step - loss: 0.0222 - acc: 0.9923 - val_loss: 0.0626 - val_acc: 0.9826
Epoch 123/200
9200/9200 [==============================] - 1s 138us/step - loss: 0.0195 - acc: 0.9921 - val_loss: 0.0625 - val_acc: 0.9817
Epoch 124/200
9200/9200 [==============================] - 1s 123us/step - loss: 0.0258 - acc: 0.9909 - val_loss: 0.0618 - val_acc: 0.9813
Epoch 125/200
9200/9200 [==============================] - 1s 121us/step - loss: 0.0214 - acc: 0.9921 - val_loss: 0.0649 - val_acc: 0.9822
Epoch 126/200
9200/9200 [=================

9200/9200 [==============================] - 1s 98us/step - loss: 0.0195 - acc: 0.9933 - val_loss: 0.0658 - val_acc: 0.9804
Epoch 179/200
9200/9200 [==============================] - 1s 112us/step - loss: 0.0198 - acc: 0.9928 - val_loss: 0.0650 - val_acc: 0.9813
Epoch 180/200
9200/9200 [==============================] - 1s 93us/step - loss: 0.0174 - acc: 0.9928 - val_loss: 0.0666 - val_acc: 0.9787
Epoch 181/200
9200/9200 [==============================] - 1s 105us/step - loss: 0.0179 - acc: 0.9936 - val_loss: 0.0629 - val_acc: 0.9796
Epoch 182/200
9200/9200 [==============================] - 1s 110us/step - loss: 0.0177 - acc: 0.9938 - val_loss: 0.0666 - val_acc: 0.9796
Epoch 183/200
9200/9200 [==============================] - 1s 138us/step - loss: 0.0149 - acc: 0.9955 - val_loss: 0.0664 - val_acc: 0.9796
Epoch 184/200
9200/9200 [==============================] - 1s 126us/step - loss: 0.0194 - acc: 0.9928 - val_loss: 0.0680 - val_acc: 0.9804
Epoch 185/200
9200/9200 [==================

In [7]:
sgdr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle')
from keras.callbacks import ModelCheckpoint
filepath="weights_ZONFvsS_sgd_tests.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

clf_ann.compile(optimizer=keras.optimizers.SGD(decay=1e-4, momentum=0.9),
                loss='binary_crossentropy',
                metrics=['accuracy'])
clf_ann.fit(X_train, y_train,
            batch_size = 256,
            epochs = 30,
            validation_data=(X_test,y_test),
            callbacks=[sgdr,checkpoint])
# Predicting the Test set results
y_pred = clf_ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(classification_report(y_test, y_pred, target_names=['Seizure', 'Non-Seizure']))

Train on 9200 samples, validate on 2300 samples
Epoch 1/10
9200/9200 [==============================] - 1s 134us/step - loss: 0.0158 - acc: 0.9938 - val_loss: 0.0689 - val_acc: 0.9796

Epoch 00001: val_acc improved from -inf to 0.97957, saving model to weights_ZONFvsS_sgd_tests.best.h5
Epoch 2/10
9200/9200 [==============================] - 1s 117us/step - loss: 0.0149 - acc: 0.9951 - val_loss: 0.0685 - val_acc: 0.9796

Epoch 00002: val_acc did not improve
Epoch 3/10
9200/9200 [==============================] - 1s 104us/step - loss: 0.0169 - acc: 0.9937 - val_loss: 0.0684 - val_acc: 0.9796

Epoch 00003: val_acc did not improve
Epoch 4/10
9200/9200 [==============================] - 1s 113us/step - loss: 0.0155 - acc: 0.9943 - val_loss: 0.0684 - val_acc: 0.9796

Epoch 00004: val_acc did not improve
Epoch 5/10
9200/9200 [==============================] - 1s 103us/step - loss: 0.0148 - acc: 0.9950 - val_loss: 0.0683 - val_acc: 0.9796

Epoch 00005: val_acc did not improve
Epoch 6/10
9200/9

In [8]:
from imblearn.over_sampling import SMOTE
X_s = dataset.iloc[:,1:179].values
y_s = tgt.iloc[:].values

In [9]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s,test_size = 0.2)
sm = SMOTE(ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train_s, y_train_s)

In [11]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc',
                                           min_delta=0,
                                           patience=10000,
                                           verbose=1,
                                           mode='auto')
# Initialising the ANN
clf_ann = Sequential()

# Adding the input layer and the first hidden layer
clf_ann.add(Dense(activation="relu", 
                  units=178,
                  input_dim=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the second hidden layer
clf_ann.add(Dense(activation="relu",
                  units=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the output layer
clf_ann.add(Dense(units=1,
                  kernel_initializer='glorot_normal',
                  activation="sigmoid"))

# Compiling the ANN
clf_ann.compile(optimizer=keras.optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
# loss = 'categorical_crossentropy'
# Fitting the ANN to the Training set
clf_ann.fit(X_train_s, y_train_s,
               batch_size = 256,
               epochs = 200,
               validation_data=(X_test_s,y_test_s),
               callbacks=[early_stop])

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = clf_ann.predict(X_test_s)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
#cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test_s, y_pred, target_names=['Seizure', 'Non-Seizure']))

#df_cm = pd.DataFrame(cm, range(3), range(3))
#sn.heatmap(df_cm, annot=True,fmt='g',cmap ='Blues')# font size

Train on 9200 samples, validate on 2300 samples
Epoch 1/200
9200/9200 [==============================] - 2s 179us/step - loss: 0.2240 - acc: 0.9388 - val_loss: 0.1516 - val_acc: 0.9517
Epoch 2/200
9200/9200 [==============================] - 1s 107us/step - loss: 0.1345 - acc: 0.9597 - val_loss: 0.1167 - val_acc: 0.9565
Epoch 3/200
9200/9200 [==============================] - 1s 108us/step - loss: 0.1164 - acc: 0.9658 - val_loss: 0.1059 - val_acc: 0.9617
Epoch 4/200
9200/9200 [==============================] - 1s 105us/step - loss: 0.1036 - acc: 0.9680 - val_loss: 0.0937 - val_acc: 0.9643
Epoch 5/200
9200/9200 [==============================] - 1s 109us/step - loss: 0.0991 - acc: 0.9660 - val_loss: 0.1075 - val_acc: 0.9587
Epoch 6/200
9200/9200 [==============================] - 1s 110us/step - loss: 0.0996 - acc: 0.9653 - val_loss: 0.0981 - val_acc: 0.9635
Epoch 7/200
9200/9200 [==============================] - 1s 105us/step - loss: 0.0905 - acc: 0.9662 - val_loss: 0.0846 - val_acc: 

9200/9200 [==============================] - 1s 102us/step - loss: 0.0386 - acc: 0.9853 - val_loss: 0.0697 - val_acc: 0.9743
Epoch 61/200
9200/9200 [==============================] - 1s 108us/step - loss: 0.0403 - acc: 0.9838 - val_loss: 0.0656 - val_acc: 0.9757
Epoch 62/200
9200/9200 [==============================] - 1s 125us/step - loss: 0.0437 - acc: 0.9836 - val_loss: 0.0693 - val_acc: 0.9743
Epoch 63/200
9200/9200 [==============================] - 1s 140us/step - loss: 0.0393 - acc: 0.9833 - val_loss: 0.0698 - val_acc: 0.9743
Epoch 64/200
9200/9200 [==============================] - 1s 132us/step - loss: 0.0388 - acc: 0.9854 - val_loss: 0.0796 - val_acc: 0.9735
Epoch 65/200
9200/9200 [==============================] - 1s 128us/step - loss: 0.0422 - acc: 0.9840 - val_loss: 0.0728 - val_acc: 0.9735
Epoch 66/200
9200/9200 [==============================] - 1s 124us/step - loss: 0.0374 - acc: 0.9863 - val_loss: 0.0733 - val_acc: 0.9743
Epoch 67/200
9200/9200 [=======================

9200/9200 [==============================] - 1s 111us/step - loss: 0.0302 - acc: 0.9886 - val_loss: 0.0711 - val_acc: 0.9761
Epoch 120/200
9200/9200 [==============================] - 1s 122us/step - loss: 0.0265 - acc: 0.9898 - val_loss: 0.0737 - val_acc: 0.9757
Epoch 121/200
9200/9200 [==============================] - 1s 111us/step - loss: 0.0290 - acc: 0.9890 - val_loss: 0.0682 - val_acc: 0.9778
Epoch 122/200
9200/9200 [==============================] - 1s 120us/step - loss: 0.0327 - acc: 0.9882 - val_loss: 0.0744 - val_acc: 0.9761
Epoch 123/200
9200/9200 [==============================] - 1s 117us/step - loss: 0.0293 - acc: 0.9893 - val_loss: 0.0709 - val_acc: 0.9774
Epoch 124/200
9200/9200 [==============================] - 1s 110us/step - loss: 0.0337 - acc: 0.9890 - val_loss: 0.0755 - val_acc: 0.9761
Epoch 125/200
9200/9200 [==============================] - 1s 92us/step - loss: 0.0297 - acc: 0.9891 - val_loss: 0.0716 - val_acc: 0.9765
Epoch 126/200
9200/9200 [=================

9200/9200 [==============================] - 1s 123us/step - loss: 0.0260 - acc: 0.9903 - val_loss: 0.0759 - val_acc: 0.9783
Epoch 179/200
9200/9200 [==============================] - 1s 119us/step - loss: 0.0267 - acc: 0.9901 - val_loss: 0.0795 - val_acc: 0.9783
Epoch 180/200
9200/9200 [==============================] - 1s 118us/step - loss: 0.0261 - acc: 0.9903 - val_loss: 0.0770 - val_acc: 0.9778
Epoch 181/200
9200/9200 [==============================] - 1s 124us/step - loss: 0.0292 - acc: 0.9897 - val_loss: 0.0763 - val_acc: 0.9783
Epoch 182/200
9200/9200 [==============================] - 1s 119us/step - loss: 0.0283 - acc: 0.9895 - val_loss: 0.0763 - val_acc: 0.9765
Epoch 183/200
9200/9200 [==============================] - 1s 119us/step - loss: 0.0269 - acc: 0.9912 - val_loss: 0.0751 - val_acc: 0.9778
Epoch 184/200
9200/9200 [==============================] - 1s 131us/step - loss: 0.0276 - acc: 0.9905 - val_loss: 0.0714 - val_acc: 0.9791
Epoch 185/200
9200/9200 [================

In [14]:
sgdr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle')
from keras.callbacks import ModelCheckpoint
filepath="weights_ZONFvsS_sgd_smote_tests_new.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

clf_ann.compile(optimizer=keras.optimizers.SGD(decay=1e-4, momentum=0.9),
                loss='binary_crossentropy',
                metrics=['accuracy'])
clf_ann.fit(X_train_s, y_train_s,
            batch_size = 256,
            epochs = 30,
            validation_data=(X_test_s,y_test_s),
            callbacks=[sgdr,checkpoint])
# Predicting the Test set results
y_pred = clf_ann.predict(X_test_s)
y_pred = (y_pred > 0.5)
print(classification_report(y_test_s, y_pred, target_names=['Seizure', 'Non-Seizure']))

Train on 9200 samples, validate on 2300 samples
Epoch 1/30
9200/9200 [==============================] - 1s 154us/step - loss: 0.0217 - acc: 0.9925 - val_loss: 0.4963 - val_acc: 0.7996

Epoch 00001: val_acc improved from -inf to 0.79957, saving model to weights_ZONFvsS_sgd_smote_tests_new.best.h5
Epoch 2/30
9200/9200 [==============================] - 1s 105us/step - loss: 0.0233 - acc: 0.9924 - val_loss: 0.1876 - val_acc: 0.9287

Epoch 00002: val_acc improved from 0.79957 to 0.92870, saving model to weights_ZONFvsS_sgd_smote_tests_new.best.h5
Epoch 3/30
9200/9200 [==============================] - 1s 111us/step - loss: 0.0198 - acc: 0.9929 - val_loss: 0.0937 - val_acc: 0.9700

Epoch 00003: val_acc improved from 0.92870 to 0.97000, saving model to weights_ZONFvsS_sgd_smote_tests_new.best.h5
Epoch 4/30
9200/9200 [==============================] - 1s 123us/step - loss: 0.0227 - acc: 0.9914 - val_loss: 0.0631 - val_acc: 0.9796

Epoch 00004: val_acc improved from 0.97000 to 0.97957, saving 